# FP-Growth Modelling: Khai thác tập mục phổ biến và sinh luật

**Mục đích:** Khai thác frequent itemsets bằng FP-Growth và sinh luật kết hợp, sau đó lọc và trực quan hóa các luật quan tâm (theo lift / confidence / support).

**Tổng quan:** Chạy các bước: (1) nạp `basket_bool`, (2) chạy FP-Growth để lấy frequent itemsets, (3) sinh luật theo metric và threshold, (4) lọc & sắp xếp luật, (5) lưu và vẽ các biểu đồ để phân tích.

_Hướng dẫn nhanh:_ Thay đổi tham số ở cell `PARAMETERS` (ví dụ `MIN_SUPPORT`, `METRIC`, `FILTER_*`) rồi chạy tuần tự notebook để nhận kết quả mới.

In [ ]:
# PARAMETERS (for papermill)

BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Đường dẫn lưu file luật kết hợp sau khi lọc
RULES_OUTPUT_PATH = "data/processed/rules_fpgrowth_filtered.csv"

# Đường dẫn lưu frequent itemsets (để đối chiếu / debug / report)
FREQ_OUTPUT_PATH = "data/processed/frequent_itemsets_fpgrowth.csv"

# Tham số cho bước khai thác tập mục phổ biến (frequent itemsets)
MIN_SUPPORT = 0.01     # ngưỡng support tối thiểu
MAX_LEN = 3            # độ dài tối đa của itemset (số sản phẩm trong 1 tập)

# Tham số cho bước sinh luật
METRIC = "lift"        # chỉ số dùng để generate rules: 'support' / 'confidence' / 'lift'
MIN_THRESHOLD = 1.0    # ngưỡng tối thiểu cho METRIC

# Tham số lọc luật sau khi generate
FILTER_MIN_SUPPORT = 0.01
FILTER_MIN_CONF = 0.3
FILTER_MIN_LIFT = 1.2
FILTER_MAX_ANTECEDENTS = 2
FILTER_MAX_CONSEQUENTS = 1

# Số lượng luật top để vẽ biểu đồ
TOP_N_RULES = 20

# Bật/tắt các biểu đồ matplotlib
PLOT_TOP_LIFT = True
PLOT_TOP_CONF = True
PLOT_SCATTER = True
PLOT_NETWORK = True

# Bật/tắt biểu đồ HTML tương tác (Plotly)
PLOT_PLOTLY_SCATTER = True


## Tham số (PARAMETERS)

**Giải thích các tham số chính:**

- `BASKET_BOOL_PATH`: đường dẫn tới ma trận boolean (mỗi hàng là một giỏ hàng).
- `MIN_SUPPORT`, `MAX_LEN`: ngưỡng support tối thiểu và độ dài tối đa của itemset.
- `METRIC`, `MIN_THRESHOLD`: metric để sinh luật (`lift`, `confidence`, `support`) và ngưỡng tối thiểu cho metric đó.
- `FILTER_*`: các ngưỡng để lọc luật sau khi sinh (support, confidence, lift) và giới hạn kích thước antecedents/consequents.
- `TOP_N_RULES`: số luật hàng đầu để vẽ biểu đồ.

> Điều chỉnh các tham số này để khám phá tập luật khác nhau hoặc tối ưu tốc độ/độ chính xác.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Biểu đồ tương tác HTML
import plotly.express as px

from apriori_library import FPGrowthMiner, DataVisualizer


## Thiết lập môi trường & Imports

Cell này thực hiện:

- Kích hoạt autoreload để tiện phát triển.
- Xác định `project_root` và thêm `src` vào `sys.path` để import `FPGrowthMiner` và `DataVisualizer` từ `src/apriori_library.py`.
- Nạp các thư viện cần thiết: pandas, matplotlib, seaborn, networkx, plotly, v.v.

> Nếu import `apriori_library` lỗi, kiểm tra xem `src` đã được thêm vào `sys.path` chính xác hay chưa và file `apriori_library.py` tồn tại.

In [ ]:
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)
print("basket_bool shape:", basket_bool.shape)
basket_bool.head()

## Nạp dữ liệu (basket_bool)

- `basket_bool` được đọc từ `BASKET_BOOL_PATH` (Parquet) và có dạng boolean matrix (mỗi cột là 1 sản phẩm).
- Kiểm tra `basket_bool.shape` và một vài hàng đầu bằng `head()` để xác thực dữ liệu.
- Nếu không có file, chạy `basket_preparation.ipynb` để tạo `basket_bool.parquet`.

> Lưu ý: Ma trận này là input trực tiếp cho cả Apriori và FP-Growth (True = sản phẩm có trong giỏ).

In [ ]:
miner = FPGrowthMiner(basket_bool=basket_bool)

start = time.perf_counter()
frequent_itemsets = miner.run(min_support=MIN_SUPPORT, use_colnames=True, max_len=MAX_LEN)
end = time.perf_counter()

print(f"FP-Growth frequent itemsets: {len(frequent_itemsets)}")
print(f"Runtime (frequent itemsets): {end - start:.4f} seconds")
frequent_itemsets.head()

## Khai thác frequent itemsets (FP-Growth)

- `FPGrowthMiner.run(...)` được gọi với `MIN_SUPPORT` và `MAX_LEN` để tìm frequent itemsets.
- Cell in báo số lượng frequent itemsets và thời gian chạy — dùng để so sánh hiệu năng.
- Thử nghiệm với `MIN_SUPPORT` khác nhau có thể ảnh hưởng mạnh đến số lượng itemset và thời gian tính toán.

> Nếu runtime quá chậm cho `MIN_SUPPORT` nhỏ, thử tăng `MIN_SUPPORT` để giảm không gian tìm kiếm.

In [ ]:
start = time.perf_counter()
if hasattr(miner, "association_rules"):
    rules = miner.association_rules(
        frequent_itemsets=frequent_itemsets,
        metric=METRIC,
        min_threshold=MIN_THRESHOLD
    )
else:
    rules = miner.generate_rules(
        frequent_itemsets=frequent_itemsets,
        min_confidence=MIN_THRESHOLD if METRIC == "confidence" else 0.0,
        metric=METRIC
    )

end = time.perf_counter()

print(f"Generated rules: {len(rules)}")
print(f"Runtime (generate rules): {end - start:.4f} seconds")
rules.head()

## Sinh luật (Rules generation)

- Sử dụng `miner.association_rules(...)` nếu tồn tại, ngược lại fallback sang `miner.generate_rules(...)`.
- `METRIC` và `MIN_THRESHOLD` điều khiển loại luật được sinh (ví dụ `METRIC='lift'` và `MIN_THRESHOLD=1.0` sẽ sinh luật có lift >= 1.0).
- Cell này in số luật được tạo và thời gian cho bước sinh luật — thông tin hữu ích để chọn tham số.

> Kiểm tra `rules.head()` để xem cấu trúc cột (`antecedents`, `consequents`, `support`, `confidence`, `lift`).

In [ ]:
rules_filtered = rules.copy()

# Lọc theo support/confidence/lift
rules_filtered = rules_filtered[
    (rules_filtered["support"] >= FILTER_MIN_SUPPORT) &
    (rules_filtered["confidence"] >= FILTER_MIN_CONF) &
    (rules_filtered["lift"] >= FILTER_MIN_LIFT)
].copy()

# Lọc theo độ dài antecedents/consequents
rules_filtered["antecedent_len"] = rules_filtered["antecedents"].apply(lambda s: len(s))
rules_filtered["consequent_len"] = rules_filtered["consequents"].apply(lambda s: len(s))

rules_filtered = rules_filtered[
    (rules_filtered["antecedent_len"] <= FILTER_MAX_ANTECEDENTS) &
    (rules_filtered["consequent_len"] <= FILTER_MAX_CONSEQUENTS)
].copy()

rules_filtered = rules_filtered.sort_values(["lift", "confidence", "support"], ascending=False).reset_index(drop=True)

print("Rules after filtering:", len(rules_filtered))
rules_filtered.head(10)

## Lọc luật & Sắp xếp

- `rules_filtered` được tạo bằng cách: lọc theo `support`, `confidence`, `lift` (theo `FILTER_*`) và giới hạn kích thước antecedents/consequents (theo `FILTER_MAX_*`).
- Sau đó sắp xếp theo `lift`, `confidence`, `support` để lấy các luật ‘‘tốt nhất’’.
- Kiểm tra `rules_filtered.head(10)` để xem các luật hàng đầu.

> Gợi ý: Bạn có thể thay đổi `FILTER_*` để cân bằng giữa số lượng và chất lượng luật.

In [ ]:
os.makedirs(os.path.dirname(RULES_OUTPUT_PATH), exist_ok=True)

frequent_itemsets.to_csv(FREQ_OUTPUT_PATH, index=False)
rules_filtered.to_csv(RULES_OUTPUT_PATH, index=False)

print("Saved frequent itemsets to:", FREQ_OUTPUT_PATH)
print("Saved filtered rules to:", RULES_OUTPUT_PATH)

## Lưu kết quả & Trực quan hoá

- Các file được lưu:
  - `FREQ_OUTPUT_PATH`: frequent itemsets (CSV)
  - `RULES_OUTPUT_PATH`: luật đã lọc (CSV)
- `DataVisualizer` sẽ sử dụng `rules_filtered` để vẽ các biểu đồ: top lift, top confidence, scatter (support vs confidence), và network graph của các luật hàng đầu.

**Giải thích nhanh các biểu đồ:**
- Top Lift / Top Confidence: cho thấy luật có lift/confidence cao nhất — thường là luật quan tâm để phân tích (nhưng lift cao không đồng nghĩa hữu ích nếu support thấp).
- Scatter (support vs confidence): giúp cân nhắc giữa phổ biến (support) và độ tin cậy (confidence) của luật.
- Network graph: trực quan mối quan hệ giữa antecedents và consequents trên các luật top.

> Gợi ý: Thay đổi `TOP_N_RULES` và các flag `PLOT_*` để điều chỉnh đầu ra đồ họa.

In [ ]:
viz = DataVisualizer()

top_rules = rules_filtered.head(TOP_N_RULES).copy()

if PLOT_TOP_LIFT and not top_rules.empty:
    viz.plot_top_rules_by_metric(top_rules, metric="lift", top_n=TOP_N_RULES, title="Top Rules by Lift (FP-Growth)")

if PLOT_TOP_CONF and not top_rules.empty:
    viz.plot_top_rules_by_metric(top_rules, metric="confidence", top_n=TOP_N_RULES, title="Top Rules by Confidence (FP-Growth)")

if PLOT_SCATTER and not rules_filtered.empty:
    viz.plot_rules_scatter(rules_filtered, title="Rules Scatter (FP-Growth): Support vs Confidence")

if PLOT_NETWORK and not top_rules.empty:
    viz.plot_rules_network(top_rules, title="Rules Network Graph (FP-Growth)")